In [1]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
# !pip install nltk

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached regex-2025.11.3-cp313-cp313-macosx_10_13_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached regex-2025.11.3-cp313-cp313-macosx_10_13_x86_64.whl (291 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [nltk]2/3 [nltk]]


In [3]:
import pandas as pd
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    print("Downloading necessary NLTK data...")
    nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'omw-1.4', 'averaged_perceptron_tagger_eng'], quiet=True)
    print("NLTK data download complete.")

NLTK data download complete.


In [10]:
# Import functions from ../scripts
import sys
sys.path.append('../scripts')
from utils import load_data, get_wordnet_pos, extract_features,clean_and_tokenize,lemmatize_tokens,preprocess_data, vectorize_data

In [ ]:
# !pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [11]:
if __name__ == '__main__':
    TEXT_COLUMN = 'Tweet'

    print("--- 1. Loading Data ---")
    df = load_data(text_column=TEXT_COLUMN)

    if df is None:
        print("Exiting script due to data loading error.")
        exit()

    df_sample = df.head(100).copy()

    print(f"Successfully loaded {len(df)} rows. Processing a sample of {len(df_sample)} rows.")
    print("\n--- Original Data Sample (First 5 rows) ---")
    print(df_sample[[TEXT_COLUMN]].head().to_markdown(index=False))

    print("\n--- 2. Applying Preprocessing Pipeline ---")
    processed_df = preprocess_data(df_sample, text_column=TEXT_COLUMN)

    print("\n--- 3. Processed Data Sample (First 5 rows) ---")
    print(processed_df[[TEXT_COLUMN, 'hashtags', 'mentions', 'processed_text']].head().to_markdown(index=False))

    print("\n--- 4. Applying TF-IDF Vectorization (sublinear_tf=True) ---")
    tfidf_df, vectorizer = vectorize_data(processed_df)

    print(f"\nTotal features (vocabulary size): {len(vectorizer.get_feature_names_out())}")

    print("\n--- 5. TF-IDF Vectorization Sample (First 5 rows, first 10 features) ---")

    feature_names = vectorizer.get_feature_names_out()[:10]
    tfidf_sample_output = tfidf_df.iloc[:5, :10]
    tfidf_sample_output.columns = feature_names
    tfidf_sample_output.index = [f"Doc {i+1}" for i in range(5)]

    print(tfidf_sample_output.to_markdown())

    print("\n--- Script Execution Complete ---")

--- 1. Loading Data ---
Successfully loaded 73996 rows. Processing a sample of 100 rows.

--- Original Data Sample (First 5 rows) ---
| Tweet                                                     |
|:----------------------------------------------------------|
| im getting on borderlands and i will murder you all ,     |
| I am coming to the borders and I will kill you all,       |
| im getting on borderlands and i will kill you all,        |
| im coming on borderlands and i will murder you all,       |
| im getting on borderlands 2 and i will murder you me all, |

--- 2. Applying Preprocessing Pipeline ---

--- 3. Processed Data Sample (First 5 rows) ---
| Tweet                                                     | hashtags   | mentions   | processed_text                                      |
|:----------------------------------------------------------|:-----------|:-----------|:----------------------------------------------------|
| im getting on borderlands and i will murder you all ,

In [19]:
tfidf_sample_output

,10,20,2010,2011,2wmmip5,45,5wf9jg,610,ability,about
Doc 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Doc 5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
processed_df

,ID,Entity,Sentiment,Tweet,hashtags,mentions,tokens,lemmas,processed_text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,,,"[im, getting, on, borderlands, and, i, will, m...","[im, get, on, borderland, and, i, will, murder...",im get on borderland and i will murder you all
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,,,"[I, am, coming, to, the, borders, and, I, will...","[I, be, come, to, the, border, and, I, will, k...",I be come to the border and I will kill you all
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,,,"[im, getting, on, borderlands, and, i, will, k...","[im, get, on, borderland, and, i, will, kill, ...",im get on borderland and i will kill you all
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,,,"[im, coming, on, borderlands, and, i, will, mu...","[im, come, on, borderland, and, i, will, murde...",im come on borderland and i will murder you all
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,,,"[im, getting, on, borderlands, 2, and, i, will...","[im, get, on, borderland, 2, and, i, will, mur...",im get on borderland 2 and i will murder you m...
...,...,...,...,...,...,...,...,...,...
95,2417,Borderlands,Negative,Grounded almost looked pretty cool even despit...,,,"[Grounded, almost, looked, pretty, cool, even,...","[Grounded, almost, look, pretty, cool, even, d...",Grounded almost look pretty cool even despite ...
96,2417,Borderlands,Negative,Grounded looked cool despite the borderline un...,,,"[Grounded, looked, cool, despite, the, borderl...","[Grounded, look, cool, despite, the, borderlin...",Grounded look cool despite the borderline unfu...
97,2417,Borderlands,Negative,"Grosskreutz looked pretty cool, even despite t...",,,"[Grosskreutz, looked, pretty, cool, even, desp...","[Grosskreutz, look, pretty, cool, even, despit...",Grosskreutz look pretty cool even despite the ...
98,2417,Borderlands,Negative,Grounded almost looked pretty cool here despit...,,,"[Grounded, almost, looked, pretty, cool, here,...","[Grounded, almost, look, pretty, cool, here, d...",Grounded almost look pretty cool here despite ...


In [ ]:
# !pip install seaborn

In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

features = ['Tweet', 'hashtags', 'mentions', 'tokens', 'lemmas','processed_text']
corr_matrix = processed_df[features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix,
            annot=True,
            fmt=".2f",
            cmap='coolwarm',
            cbar=True,
            square=True,
            linewidths=.5,
            linecolor='black')
plt.title('Correlation Matrix of Tweets Features and Sentiment')
plt.show()

print("\nCorrelation with Sentiment:")
print(corr_matrix['Sentiment'].sort_values(ascending=False))

ValueError: could not convert string to float: 'im getting on borderlands and i will murder you all ,'

In [ ]:
import zipfile
import os

zip_file_path = '/content/cardio_train.csv.zip'

extraction_dir = '/content/'

os.makedirs(extraction_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

print(f"'{zip_file_path}' unzipped to '{extraction_dir}'")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc',
            'smoke', 'alco', 'active', 'gender', 'cardio']

# Ensure all specified features exist in the DataFrame
missing_features = [f for f in features if f not in processed_df.columns]
if missing_features:
    print(f"Warning: The following features are missing from processed_df: {missing_features}")
    # Filter out missing features for correlation calculation
    available_features = [f for f in features if f in processed_df.columns]
else:
    available_features = features

corr_matrix = processed_df[available_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix,
            annot=True,
            fmt=".2f",
            cmap='coolwarm',
            cbar=True,
            square=True,
            linewidths=.5,
            linecolor='black')
plt.title('Correlation Matrix of Clinical Features and Cardiovascular Disease')
plt.show()

print("\nCorrelation with Cardiovascular Disease (cardio):")
if 'cardio' in available_features:
    print(corr_matrix['cardio'].sort_values(ascending=False))
else:
    print(" 'cardio' feature is not available for correlation calculation.")

In [ ]:
df_eda = processed_df.copy()
df_eda['age_years'] = (df_eda['age'] / 365.25).round().astype(int)

min_age = df_eda['age_years'].min()
max_age = df_eda['age_years'].max()
bins = range(min_age, max_age + 6, 5)

df_eda['age_bin'] = pd.cut(df_eda['age_years'],
                           bins=bins,
                           right=False,
                           labels=[f'{i}-{i+4}' for i in bins[:-1]])

prevalence_by_age = df_eda.groupby('age_bin', observed=False)['cardio'].agg(
    total_patients=('size'),
    disease_prevalence=('mean')
).reset_index()

print("Disease Prevalence by 5-Year Age Bin:")
print(prevalence_by_age)

plt.figure(figsize=(10, 6))
sns.barplot(x='age_bin', y='disease_prevalence', data=prevalence_by_age, palette='viridis')
plt.title('Cardiovascular Disease Prevalence by 5-Year Age Group')
plt.xlabel('Age Bin (Years)')
plt.ylabel('Disease Prevalence (Ratio)')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
def categorize_bp(ap_hi, ap_lo):
    if ap_hi < 120 and ap_lo < 80:
        return 'Normal'
    elif (ap_hi >= 140 or ap_lo >= 90):
        return 'Stage 2 Hypertension'
    elif (ap_hi >= 130 and ap_hi < 140) or (ap_lo >= 80 and ap_lo < 90):
        return 'Stage 1 Hypertension'
    elif (ap_hi >= 120 and ap_hi < 130) and (ap_lo < 80):
        return 'Elevated/Pre-Hypertension'
    else:
        return 'Stage 1 Hypertension'

df_eda['bp_category'] = df_eda.apply(
    lambda row: categorize_bp(row['ap_hi'], row['ap_lo']),
    axis=1
)

plt.figure(figsize=(8, 6))
bp_counts = df_eda['bp_category'].value_counts().sort_index()
sns.barplot(x=bp_counts.index, y=bp_counts.values, palette='plasma')
plt.title('Patient Distribution by Blood Pressure Category')
plt.xlabel('Blood Pressure Category')
plt.ylabel('Number of Patients')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
import plotly.express as px

fig = px.scatter(df_eda,
                 x='age_years',
                 y='weight',
                 color='cardio',
                 color_discrete_map={0: 'blue', 1: 'red'},
                 title='Age vs. Weight by Cardiovascular Disease Status',
                 labels={'cardio': 'Cardio Disease (1=Yes, 0=No)'},
                 hover_data=['ap_hi', 'ap_lo', 'cholesterol'])

fig.show()

In [ ]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df_eda['cholesterol'], df_eda['cardio'])

chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print("Chi-Square Test: Cholesterol vs. Cardiovascular Disease")
print(f"Chi-square statistic: {chi2:.2f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Conclusion: The relationship between cholesterol level and cardiovascular disease is statistically significant.")
else:
    print("Conclusion: The relationship is NOT statistically significant.")